# How to reproduce a Neptune run

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/how-to-guides/reproduce-run/notebooks/reproduce_run.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>
<a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/how-to-guides/reproduce-run/notebooks/reproduce_run.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a>
<a target="_blank" href="https://app.neptune.ai/o/common/org/showroom/e/SHOW-30720/all"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a>
<a target="_blank" href="https://docs.neptune.ai/tutorials/reproducing_runs/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction
When building ML models for research or production, it's crucial to be able to reproduce a run to validate its results and performance. With Neptune, you can reproduce any run by retrieving the same metadata - such as hyperparameters, data, and code version.

In this guide, we'll show you how to re-open an existing Neptune run to retrieve the metadata required for reproducing it. 

## Before you start

Make sure that you have:
* [Python 3.7+ installed](https://www.python.org/downloads/),
* [Basic familiarity with Neptune (create run and log metadata to it)](https://docs.neptune.ai/usage/#getting-started)

In [ ]:
! pip install -U neptune torch torchvision

**Note**: To log or retrieve metadata from Neptune, you need the project name and the API token.

To make this example easy to follow, we'll log the metadata to the public project "common/showroom" using a shared token for anonymous logging.

**(Optional)** If you want to log to your own project, you need a [Neptune account](https://app.neptune.ai/register/) and a [project](https://docs.neptune.ai/setup/creating_project).
Then you can pass the [project](https://docs.neptune.ai/setup/creating_project/#next-steps) and [api_token](https://docs.neptune.ai/setup/setting_api_token/#setting-your-api-token) arguments to the `init_run()` method or as environment variables.

In [ ]:
import os
import neptune

# (Neptune) Setting up credentials as env variables
os.environ["NEPTUNE_PROJECT"] = "common/showroom"  # You can replace this with your own project
os.environ["NEPTUNE_API_TOKEN"] = (
    neptune.ANONYMOUS_API_TOKEN
)  # You can replace this with your own token

## Step 1: Get run ID
You will get the run ID of the run you want to reproduce programmatically.

In [ ]:
# (Neptune) Fetch only inactive runs with tags "showcase-run", "reproduce" and "Basic script" from project
with neptune.init_project(mode="read-only") as project:
    runs_table_df = project.fetch_runs_table(
        state="inactive", tag=["showcase-run", "reproduce", "Basic script"]
    ).to_pandas()

# Extract the last successful run's id
old_run_id = runs_table_df[runs_table_df["sys/failed"] == False]["sys/id"].values[0]

In [ ]:
print(f"{old_run_id=}")

## Step 2: Resume old run and fetch relevant metadata from Neptune
Use the `neptune.init_run()` method to re-open an existing run using the ID you got from the previous step.

Use the `read-only` mode so the metadata previously logged to the run is not accidentally changed. Also, you can re-open a run as many times as needed.

In [ ]:
old_run = neptune.init_run(
    with_id=old_run_id,
    mode="read-only",
)

Fetch metadata (i.e., dataset and hyperparameters) needed to re-run the training. 

Precisely, you will download the hyperparameters and dataset path used in the old run to instantiate a model and dataset objects with the same configuration.

Use the [fetch()](https://docs.neptune.ai/api/field_types/#fetch_1) method to retrieve relevant metadata.

In [ ]:
# (Neptune) Fetch hyperparameters
old_run_params = old_run["config/params"].fetch()

In [ ]:
# (Neptune) Fetch dataset path
dataset_path = old_run["config/dataset/path"].fetch()

## Step 3: Create a new run
Create a new Neptune run that will be used to log metadata in the re-run session.

In [ ]:
new_run = neptune.init_run(
    tags=["reproduce", "new-run"],
)

Running this cell creates a run in Neptune, and you can log model building metadata to it.

**Click on the link above to open the run in the Neptune app.** 

For now, it is empty, but you should keep the tab open to see what happens next.

## Step 4: Log hyperparameters and dataset details from old run to new run
Now you can continue working and logging metadata to a brand new run.

You can log metadata using the Neptune client library. For details, see [What you can log and display](https://docs.neptune.ai/logging/what_you_can_log).

In [ ]:
new_run["config/params"] = old_run_params
new_run["config/dataset/path"] = dataset_path

### Load dataset and model

Dataset

In [ ]:
import torch
from torchvision import datasets, transforms

data_tfms = {
    "train": transforms.Compose(
        [
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
}

In [ ]:
trainset = datasets.CIFAR10(dataset_path, transform=data_tfms["train"], download=True)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=old_run_params["bs"], shuffle=True, num_workers=0
)

Model

In [ ]:
import torch.nn as nn


class BaseModel(nn.Module):
    def __init__(self, input_sz, hidden_dim, n_classes):
        super(BaseModel, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(input_sz, hidden_dim * 2),
            nn.ReLU(),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, n_classes),
        )

    def forward(self, input):
        x = input.view(-1, 32 * 32 * 3)
        return self.main(x)

In [ ]:
model = BaseModel(
    old_run_params["input_sz"],
    old_run_params["input_sz"],
    old_run_params["n_classes"],
)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=old_run_params["lr"])

## Step 5: Log losses and metrics

In [ ]:
for i, (x, y) in enumerate(trainloader, 0):
    optimizer.zero_grad()
    outputs = model.forward(x)
    _, preds = torch.max(outputs, 1)
    loss = criterion(outputs, y)
    acc = (torch.sum(preds == y.data)) / len(x)

    new_run["training/batch/loss"].append(loss)

    new_run["training/batch/acc"].append(acc)

    loss.backward()
    optimizer.step()

## Stop logging

Once you are done logging, stop each of the runs.

In [ ]:
old_run.stop()
new_run.stop()

## Conclusion
You learned how to:
* Re-open an old run in order to fetch the metadata needed to reproduce it.
* Use fetched metadata to parametrize a new run with the same training loop.

Visit our docs for more tutorials and guides on how to use Neptune: https://docs.neptune.ai
